In [295]:
import pandas as pd
import numpy as np
import os
import re
import json
import presto  # import prestosql python client
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter('ignore', InsecureRequestWarning)

In [296]:
uid = "_svc_acs_presto"
pwd = "em6qF7Gp8E9ECYcKx3gHZ#v$H@pzT7AX3uCJp%5wesKE%VD6gZaCB8*rZ2zNgMKdPkuetSJph2H&H2eadbfCkeB3tBbUqcM^Sj5k"

conn = presto.dbapi.connect(host='acs-exports.bi.ghdna.io',
                            port=8446,
                            user='_svc_acs_presto',
                            catalog='hive',
                            schema='default',
                            http_scheme='https',
                            verify=False,
                            auth=presto.auth.BasicAuthentication(uid, pwd))
cur = conn.cursor()

In [297]:
path = os.listdir("/ghds/cv19/analysis")
r = re.compile(r"20(11|12).{27}$")
file_names2 = list(filter(r.match, path))

In [298]:
low_reads = []
for i in file_names2:
    current = pd.read_csv(f"/ghds/cv19/analysis/{i}/pool_stats.hdr.tsv", sep='\t').iloc[:, :4]
    std_dev = current['reads_total'].std()
    mean = current['reads_total'].mean().astype('int')
    lower_bound = (mean - 2 * std_dev).astype('int')
    current['mean'] = mean
    current['lower_bound'] = lower_bound
    low_reads.append(current.query(f'reads_total < 1000000 and reads_total < {lower_bound}'))
    
df2 = pd.concat(low_reads)
    
df2['well_position'] = df2['pos_pooling'].str.split(":").str[1]
df2['date'] = df2['runid'].str[:6]
df2['flowcell'] = df2.runid.str[22:]
df2.sort_values(by='date', ascending=False, inplace=True)
df2

,runid,pos_pooling,index,reads_total,mean,lower_bound,well_position,date,flowcell
19,201212_NB552398_0053_AHHGFGBGXG,PRP200924A073:G2,CCTCGGAC_TTGCAGAT,261163,8605656,3575945,G2,201212,HHGFGBGXG
4,201211_NB552393_0049_AHHFG2BGXG,PRP200924A072:C1,GTAGGCTG_ACCCAGGT,384530,18463850,4783204,C1,201211,HHFG2BGXG
15,201209_NB552393_0046_AHHJLLBGXG,PRP200924A067:F1,AACAGACG_CTGTGTTG,325727,16241907,6407026,F1,201209,HHJLLBGXG
9,201202_NB552398_0049_AHHHT5BGXG,PRP200924A062:F1,AACAGACG_CTGTGTTG,275438,25740162,6374294,F1,201202,HHHT5BGXG
9,201125_NB552398_0048_AHHJNNBGXG,PRP200924A059:F1,AACAGACG_CTGTGTTG,399064,25893925,3926407,F1,201125,HHJNNBGXG
14,201124_NB552398_0047_AHHJM7BGXG,PRP200924A057:B7,AACGGGCT_CTCCGATC,109588,5258300,2320781,B7,201124,HHJM7BGXG
16,201124_NB552398_0047_AHHJM7BGXG,PRP200924A057:C1,GTAGGCTG_ACCCAGGT,128646,5258300,2320781,C1,201124,HHJM7BGXG
12,201114_NB552398_0043_AHH5YCBGXG,PRP200924A046:C3,CTATACCC_ACTACACG,173404,8243559,2422734,C3,201114,HH5YCBGXG
24,201114_NB552398_0043_AHH5YCBGXG,PRP200924A046:F1,AACAGACG_CTGTGTTG,280834,8243559,2422734,F1,201114,HH5YCBGXG
30,201114_NB552398_0043_AHH5YCBGXG,PRP200924A046:G3,GCTGGGCC_GAGAGATG,174506,8243559,2422734,G3,201114,HH5YCBGXG


In [299]:
for r in cur.execute('SHOW tables from acs_default_cv19.public'):
    print(r)

['accounts_permissionsplaceholder']
['accounts_userprofile']
['audit_actionauth']
['audit_additionalcontext']
['audit_auditoption']
['audit_auditrecord']
['audit_event']
['audit_proxyuserrevisionmeta']
['audit_revisionaction']
['audit_rfc']
['audit_secondsign']
['auth_group']
['auth_group_permissions']
['auth_permission']
['auth_user']
['auth_user_groups']
['auth_user_user_permissions']
['authtoken_token']
['batch_batch']
['batch_batch_consumers']
['batch_batch_operators']
['batch_batch_origins']
['batch_batch_parents']
['batch_batchmaptemplate']
['batch_flowcell']
['batch_flowcell_related_batch']
['batch_repeatbatch']
['certification_assaycertificateconfig']
['certification_assaycertificateconfig_required_certs']
['certification_certificate']
['certification_instrumentcertificateconfig']
['certification_instrumentcertificateconfig_required_certs']
['certification_personalcertification']
['certification_personaltraineerecord']
['certification_personaltrainerrecord']
['column_column']
[

In [300]:
column_names = []
for r in cur.execute('SHOW COLUMNS from acs_default_cv19.public.operation_operation'):
    column_names.append(r[0])
for r in cur.execute('SHOW COLUMNS from acs_default_cv19.public.operation_operation_operators'):
    column_names.append(r[0])
for r in cur.execute('SHOW COLUMNS from acs_default_cv19.public.auth_user'):
    column_names.append(r[0])
column_names

['created',
 'modified',
 'uuid',
 'input_data',
 'qc_status',
 'output_data',
 'state',
 'start_time',
 'end_time',
 'name',
 'stage',
 'assay',
 'is_rerun',
 'pipeline_item_id',
 'type_id',
 'is_final',
 'progress',
 'is_repeat',
 'acs_version',
 'id',
 'operation_id',
 'user_id',
 'id',
 'password',
 'last_login',
 'is_superuser',
 'username',
 'first_name',
 'last_name',
 'email',
 'is_staff',
 'is_active',
 'date_joined']

In [311]:
data = []
for r in cur.execute(
        "SELECT acs_default_cv19.public.operation_operation.input_data, acs_default_cv19.public.operation_operation.name, acs_default_cv19.public.auth_user.first_name\
         FROM acs_default_cv19.public.operation_operation \
         LEFT JOIN acs_default_cv19.public.operation_operation_operators ON acs_default_cv19.public.operation_operation.uuid = acs_default_cv19.public.operation_operation_operators.operation_id\
         LEFT JOIN acs_default_cv19.public.auth_user ON acs_default_cv19.public.operation_operation_operators.user_id = acs_default_cv19.public.auth_user.id\
         WHERE name = 'Load Sequencer' and created > timestamp '2020-09-01'\
         ORDER BY created"
):
    data.append(r)

In [313]:
column_names=['input_data', 'operation', 'first_name']
x = pd.DataFrame(data, columns=column_names)
x['flowcell'] = [json.loads(value)['data']['Flowcell ID'] for _, value in x.input_data.items()]
x

,input_data,operation,first_name,flowcell
0,"{""data"":{""Flowcell ID"":""HH3LKBGXG"",""Pool Tube ...",Load Sequencer,Mimi,HH3LKBGXG
1,"{""data"":{""Flowcell ID"":""HHJ32BGXG"",""Pool Tube ...",Load Sequencer,Mimi,HHJ32BGXG
2,"{""data"":{""Flowcell ID"":""HH5YKBGXG"",""Pool Tube ...",Load Sequencer,Mimi,HH5YKBGXG
3,"{""data"":{""Flowcell ID"":""HH3KHBGXG"",""Pool Tube ...",Load Sequencer,Mimi,HH3KHBGXG
4,"{""data"":{""Flowcell ID"":""H5FMVBGXG"",""Pool Tube ...",Load Sequencer,Tracy,H5FMVBGXG
...,...,...,...,...
91,"{""data"":{""Flowcell ID"":""HHJM5BGXG"",""Pool Tube ...",Load Sequencer,Mimi,HHJM5BGXG
92,"{""data"":{""Flowcell ID"":""HHFFMBGXG"",""Pool Tube ...",Load Sequencer,Danielle,HHFFMBGXG
93,"{""data"":{""Flowcell ID"":""HHFG2BGXG"",""Pool Tube ...",Load Sequencer,Dianna,HHFG2BGXG
94,"{""data"":{""Flowcell ID"":""HHJT3BGXG"",""Pool Tube ...",Load Sequencer,Danielle,HHJT3BGXG


In [314]:
df2.merge(x, left_on='flowcell', right_on='flowcell', how='left')

,runid,pos_pooling,index,reads_total,mean,lower_bound,well_position,date,flowcell,input_data,operation,first_name
0,201212_NB552398_0053_AHHGFGBGXG,PRP200924A073:G2,CCTCGGAC_TTGCAGAT,261163,8605656,3575945,G2,201212,HHGFGBGXG,"{""data"":{""Flowcell ID"":""HHGFGBGXG"",""Pool Tube ...",Load Sequencer,Dianna
1,201211_NB552393_0049_AHHFG2BGXG,PRP200924A072:C1,GTAGGCTG_ACCCAGGT,384530,18463850,4783204,C1,201211,HHFG2BGXG,"{""data"":{""Flowcell ID"":""HHFG2BGXG"",""Pool Tube ...",Load Sequencer,Dianna
2,201209_NB552393_0046_AHHJLLBGXG,PRP200924A067:F1,AACAGACG_CTGTGTTG,325727,16241907,6407026,F1,201209,HHJLLBGXG,"{""data"":{""Flowcell ID"":""HHJLLBGXG"",""Pool Tube ...",Load Sequencer,Khiet
3,201202_NB552398_0049_AHHHT5BGXG,PRP200924A062:F1,AACAGACG_CTGTGTTG,275438,25740162,6374294,F1,201202,HHHT5BGXG,"{""data"":{""Flowcell ID"":""HHHT5BGXG"",""Pool Tube ...",Load Sequencer,Dianna
4,201125_NB552398_0048_AHHJNNBGXG,PRP200924A059:F1,AACAGACG_CTGTGTTG,399064,25893925,3926407,F1,201125,HHJNNBGXG,"{""data"":{""Flowcell ID"":""HHJNNBGXG"",""Pool Tube ...",Load Sequencer,Danielle
5,201124_NB552398_0047_AHHJM7BGXG,PRP200924A057:B7,AACGGGCT_CTCCGATC,109588,5258300,2320781,B7,201124,HHJM7BGXG,"{""data"":{""Flowcell ID"":""HHJM7BGXG"",""Pool Tube ...",Load Sequencer,Mimi
6,201124_NB552398_0047_AHHJM7BGXG,PRP200924A057:C1,GTAGGCTG_ACCCAGGT,128646,5258300,2320781,C1,201124,HHJM7BGXG,"{""data"":{""Flowcell ID"":""HHJM7BGXG"",""Pool Tube ...",Load Sequencer,Mimi
7,201114_NB552398_0043_AHH5YCBGXG,PRP200924A046:C3,CTATACCC_ACTACACG,173404,8243559,2422734,C3,201114,HH5YCBGXG,"{""data"":{""Flowcell ID"":""HH5YCBGXG"",""Pool Tube ...",Load Sequencer,Dianna
8,201114_NB552398_0043_AHH5YCBGXG,PRP200924A046:F1,AACAGACG_CTGTGTTG,280834,8243559,2422734,F1,201114,HH5YCBGXG,"{""data"":{""Flowcell ID"":""HH5YCBGXG"",""Pool Tube ...",Load Sequencer,Dianna
9,201114_NB552398_0043_AHH5YCBGXG,PRP200924A046:G3,GCTGGGCC_GAGAGATG,174506,8243559,2422734,G3,201114,HH5YCBGXG,"{""data"":{""Flowcell ID"":""HH5YCBGXG"",""Pool Tube ...",Load Sequencer,Dianna


In [304]:
column = []
for r in cur.execute('SHOW COLUMNS from acs_default_cv19.public.operation_operation'):
    column.append(r[0])

value = []
for r in cur.execute(
        'SELECT * from acs_default_cv19.public.operation_operation'):
    value.append(r)

In [305]:
pd.DataFrame(value, columns=column).query("name=='iPCR Clean Up'")

,created,modified,uuid,input_data,qc_status,output_data,state,start_time,end_time,name,stage,assay,is_rerun,pipeline_item_id,type_id,is_final,progress,is_repeat,acs_version
60,2020-06-01 02:22:14.642 UTC,2020-06-01 02:22:14.642 UTC,5e59b49a-eac0-49cf-86f3-5d5023ebd857,"{""instr_data"":{""100% EtOH"":""20200204-SHBL2095""...",Pass,{},Finished,2020-06-01 02:22:48.000 UTC,2020-06-01 03:02:37.000 UTC,iPCR Clean Up,Extraction,Guardant COVID-19,False,27c5801b-77b9-4645-8cc8-960e47ba84f5,732d2f86-0262-4632-be7c-b9342b7c4bb2,True,100,False,1.0 vCOVID-19-9091-8b25ca2
179,2020-06-03 03:32:43.525 UTC,2020-06-03 03:32:43.525 UTC,26cdfd38-cdf0-4147-b1a6-5868ef7e8555,"{""instr_data"":{""100% EtOH"":""20200204-SHBL2095""...",Pass,{},Finished,2020-06-03 03:42:49.000 UTC,2020-06-03 04:24:09.000 UTC,iPCR Clean Up,Extraction,Guardant COVID-19,False,d6363607-451a-4134-9a7f-4999a21a472f,732d2f86-0262-4632-be7c-b9342b7c4bb2,True,100,False,1.0 vCOVID-19-9093-4de61bf
269,2020-06-18 01:13:39.009 UTC,2020-06-18 01:51:22.150 UTC,7a20e4d6-3eee-42c2-b157-952498138798,"{""instr_data"":{""100% EtOH"":""20200204-SHBL2095""...",Pass,{},Finished,2020-06-18 01:18:28.000 UTC,2020-06-18 01:21:34.000 UTC,iPCR Clean Up,Extraction,Guardant COVID-19,False,f6b9b2e7-fcbf-4220-8438-e600c01aaa0f,5945a8a0-5e35-4a44-b16a-b9ecdbccfeb7,True,100,False,1.0 vCOVID-19-9109-57cbbe15d
306,2020-06-26 21:00:09.067 UTC,2020-06-26 21:00:09.067 UTC,51fad922-cb99-4933-9493-55e9e1366c5f,"{""instr_data"":{""100% EtOH"":""20200204-SHBL2095""...",Pass,{},Finished,2020-06-26 21:40:31.000 UTC,2020-06-26 22:30:30.000 UTC,iPCR Clean Up,Extraction,Guardant COVID-19,False,5656ee0e-f884-43a9-b5cb-2ada5964b2e0,5945a8a0-5e35-4a44-b16a-b9ecdbccfeb7,True,100,False,1.0 vCOVID-19-9109-57cbbe15d
320,2020-06-29 23:40:26.590 UTC,2020-06-29 23:40:26.590 UTC,c941c425-dfb7-4831-809b-399c4b1729ad,"{""instr_data"":{""100% EtOH"":""20200204-SHBL2095""...",Pass,{},Finished,2020-06-30 00:18:46.000 UTC,2020-06-30 00:58:13.000 UTC,iPCR Clean Up,Extraction,Guardant COVID-19,False,707d1151-6c88-4438-b6f0-05e6c299b54b,5945a8a0-5e35-4a44-b16a-b9ecdbccfeb7,True,100,False,1.0 vCOVID-19-9109-57cbbe15d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8901,2020-12-10 04:12:46.765 UTC,2020-12-10 04:12:46.765 UTC,4a0758ef-ddb0-404c-8d0f-df874ee235af,"{""instr_data"":{""100% EtOH"":""20200902-SHBM0785""...",Pass,{},Finished,2020-12-10 04:17:06.000 UTC,2020-12-10 05:29:16.000 UTC,iPCR Clean Up,Enrichment,Guardant COVID-19,False,6d86d4fb-8a8c-4a27-b320-0e8c0979351f,fb1413ae-0c1c-4546-9d60-960b111f69ca,True,100,False,1.0 vG19-1.5-RLS-9226-76b2891a2
8950,2020-12-10 04:30:18.348 UTC,2020-12-10 04:30:18.348 UTC,cdd079ea-83d0-4d95-afd4-e8c323ac32e2,"{""instr_data"":{""100% EtOH"":""20200902-SHBM0785""...",Pass,{},Finished,2020-12-10 04:49:05.000 UTC,2020-12-10 05:41:21.000 UTC,iPCR Clean Up,Enrichment,Guardant COVID-19,False,0234b1cc-d26c-4af3-8cef-746a9fc00d7d,fb1413ae-0c1c-4546-9d60-960b111f69ca,True,100,False,1.0 vG19-1.5-RLS-9226-76b2891a2
8979,2020-12-11 03:54:41.189 UTC,2020-12-11 03:54:41.189 UTC,9f185c0a-57ac-47cf-85fc-96997713d3e9,"{""instr_data"":{""100% EtOH"":""20200902-SHBM0785""...",Pass,{},Finished,2020-12-11 04:28:18.000 UTC,2020-12-11 05:07:28.000 UTC,iPCR Clean Up,Enrichment,Guardant COVID-19,False,780dfb08-3fa6-42f7-8719-41f2996bb2d2,fb1413ae-0c1c-4546-9d60-960b111f69ca,True,100,False,1.0 vG19-1.5-RLS-9226-76b2891a2
9052,2020-12-12 00:58:38.930 UTC,2020-12-12 00:58:38.930 UTC,6db7ea38-1378-4db1-826e-cff43bb9458a,"{""instr_data"":{""100% EtOH"":""20200902-SHBM0785""...",Pass,{},Finished,2020-12-12 01:08:20.000 UTC,2020-12-12 02:15:00.000 UTC,iPCR Clean Up,Enrichment,Guardant COVID-19,False,a57f3bcb-9f43-4139-a395-82604ee9bee4,fb1413ae-0c1c-4546-9d60-960b111f69ca,True,100,False,1.0 vG19-1.5-RLS-9226-76b2891a2


In [112]:
x = pd.DataFrame.from_dict(json.loads(x.data.values[0]))
pd.json_normalize(x['tube_data'])

,tube_data
HBV0497,"{'ex_barcode': 'CAGTAATCCG_CCGACAGATT', 'ex_po..."
HBX1081,"{'ex_barcode': 'TCCGGTTCGA_GGATACCTTG', 'ex_po..."
HBX1097,"{'ex_barcode': 'CGTAACTCAG_GACTGACATG', 'ex_po..."
HBX1111,"{'ex_barcode': 'AGTAGGATCA_CGAAGCTCTT', 'ex_po..."
HBX1151,"{'ex_barcode': 'GAGGCTTAGA_GTCATCTCTT', 'ex_po..."
...,...
HCU0601,"{'ex_barcode': 'GGTTCTGTCA_GGTATACCTA', 'ex_po..."
Ht01679,"{'ex_barcode': 'TGGTCTTAGG_GCCTAGTGTC', 'ex_po..."
Ht01680,"{'ex_barcode': 'CTACACTCCT_ACATTGAGAC', 'ex_po..."
N2008260817,"{'ex_barcode': 'GTTGCGGTGA_CCATGGTGAA', 'ex_po..."
